In [1]:
import os

In [2]:
%pwd

'c:\\Users\\amitt\\Documents\\DataScienceProject\\research'

In [3]:
os.chdir("../")

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class ModelTrainerConfig:

    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_path: str
    alpha: float
    l1_ratio: float
    target_col: str


In [5]:
from src.datascience.constants import *
from src.datascience.utils.common import read_yaml,create_directory

In [6]:
# updating the configuration manager
class ConfigurationManager:

    def __init__(self,config_filepath=CONFIG_FILE_PATH,
                 param_file_path=PARAMS_FILE_PATH,
                 schema_file_path=SCHEMAS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.param = read_yaml(param_file_path)
        self.schema = read_yaml(schema_file_path)


        create_directory([self.config.artifacts_root])


    def get_model_trainer(self) -> ModelTrainerConfig:

        

        config = self.config.model_trainer
        # we are getting all the parameters

        params = self.param.ElasticNet

        schema = self.schema.TARGET_COLUMN
        

        create_directory([config.root_dir])

        # creating the object with all the class attributes/members/parameters

        model_trainer_config = ModelTrainerConfig    (


            root_dir = config.root_dir,
            train_data_path= config.train_data_path,
            test_data_path = config.test_data_path,
            model_path=config.model_path,
            alpha = params.alpha,
            l1_ratio = params.l1_Ratio,
            target_col = schema.name,

        )

        # returning the object
        return model_trainer_config


In [7]:
import pandas as pd
import os

from src.datascience import logger
from sklearn.linear_model import ElasticNet

import joblib

In [8]:
class ModelTrainer:

    def __init__(self,config:ModelTrainerConfig):

        self.config = config

    def train_model(self):

        train_data = pd.read_csv(self.config.train_data_path)

        test_data = pd.read_csv(self.config.test_data_path)


        train_x = train_data.drop(self.config.target_col,axis=1)
        test_x = test_data.drop(self.config.target_col,axis=1)
        train_y = train_data[self.config.target_col]
        test_y = test_data[self.config.target_col]

        lr = ElasticNet(alpha=self.config.alpha,l1_ratio=self.config.l1_ratio,random_state=42)

        lr.fit(train_x,train_y)

        joblib.dump(lr,os.path.join(self.config.root_dir,self.config.model_path))


In [9]:
try:
    config = ConfigurationManager()

    model_trainer_config = config.get_model_trainer()

    model_trainer = ModelTrainer(model_trainer_config)

    model_trainer.train_model()

except Exception as e:

    raise e



[2024-11-03 01:32:33,257 - INFO - common - yaml file : config\config.yaml loaded successfully]
the content is : {content}, for yaml file : {yaml_file}
[2024-11-03 01:32:33,257 - INFO - common - yaml file : params.yaml loaded successfully]
the content is : {content}, for yaml file : {yaml_file}
[2024-11-03 01:32:33,259 - INFO - common - yaml file : schema.yaml loaded successfully]
the content is : {content}, for yaml file : {yaml_file}
[2024-11-03 01:32:33,260 - INFO - common - directory : artifacts created successfully]
[2024-11-03 01:32:33,261 - INFO - common - directory : artifacts/model_trainer created successfully]


AttributeError: 'ModelTrainer' object has no attribute 'root_dir'